In [1]:
# Carga de librerias a usar
import pandas as pd
import numpy as np
import re
from pathlib import Path
import locale
locale.setlocale(locale.LC_ALL, 'es_ES')

'es_ES'

In [2]:
# Se carga el archivo a limpiar, en un comienzo este archivo tenia espacios vacios, con la parte comentada se corrige ese error
path = '../Datos/XXEJE_EAM___Ciclo_de_vida_de_ó_010222.xls'
with open(path, 'rt', encoding='utf-8') as myfile:
    data = myfile.read()#.replace('</span><br/><span class="c62">', ' ').replace('</span><br/><br/><span class="c62">', ' ').replace('</span><br/><br/><br/><span class="c62">', ' ')

In [3]:
# Ahora se crea el DataFrame para trabajar
df_ori = pd.read_html(data, header=0)
df_cvo = pd.DataFrame(df_ori[0])

In [4]:
# Aca se eliminan los duplicados, y se crean columnas que se llenaran mas adelante, ademas, se separa la informacion de la columna 'Grupo de Activos' y se completa la informacion del tipo de OT, principal o secundaria
df_cvo = pd.DataFrame(df_ori[0])

df_cvo.drop_duplicates(inplace = True)
df_cvo.insert(loc=2, column='PPU_MMU', value=None)
df_cvo.insert(loc=5, column='Tipo Ot', value='Principal')
df_cvo.insert(loc=6, column='Sistema', value=None)

sist = ['Grupo Motopropulsor', 'Alimentacion De Combustible', 'Frenos', 'Rodadura', 'Direccion y Conduccion', 'Suspencion', 'Generacion e Iluminacion', 'Indicacion y Registro', 'Anfibio y Vadeo', 'Guiado de Armamento', 'Observacion y Control de Fuego', 'Armamento Principal', 'No Identificado 13', 'Plataforma Hidraulica', 'Plataforma Estructural', 'Equipamento Zapador / Recuperador', 'Extincion de Incendios', 'Acondicionamiento de Aire', 'Lanza Fumigenos', 'Estructura', 'Equipamiento Estructura', 'No Identificado 22', 'Comunicaciones']

for i, sis in enumerate(sist):
    df_cvo['Sistema'] = np.where(((df_cvo['Grupo De Activos']=='GR_SIS.02.0'+str(i+1).zfill(2)) | (df_cvo['Grupo De Activos'].str.contains('GR_SUB.02.0'+str(i+1).zfill(2)))), sis, df_cvo['Sistema'])
    
df_cvo['Numero Ot Principal'] = np.where(df_cvo['Numero Ot Principal'].str.contains('No tiene'),None,df_cvo['Numero Ot Principal'])
df_cvo['Tipo Ot'] = np.where(df_cvo['Numero Ot Principal'].isnull(),df_cvo['Tipo Ot'],'Secundaria')

In [5]:
# En esta celda se completa la infora de la columna 'PPU_MMU', se separa la informacion de la columna 'Descripcion Activo', donde su primer entrada es la patente o mmu del vehiculo. Se ocupan estos try and except, porque algunos vehiculos no tienen patente o mmu, y se debe dejar en blanco
df_cvo['PPU_MMU'] = df_cvo['Descripcion Activo'].str.split('_', expand = True)[0]
b=0
for i, n in enumerate(df_cvo['PPU_MMU']):
    try:
        if not (re.match(r'[a-zA-Z0-9]{1,4}-[a-zA-Z0-9]{1,4}$', n)):
            m = df_cvo.loc[i,'Numero Activo'][4:]
            df_cvo.loc[i,'PPU_MMU'] = m # df_cvo.loc[i,'Numero Activo'][4:]
    except:
        b=1
for i, n in enumerate(df_cvo['PPU_MMU']):
    try:
        if not (re.match(r'[a-zA-Z0-9]{1,4}-[a-zA-Z0-9]{1,4}$', n)):
            df_cvo.loc[i,'PPU_MMU'] = df_cvo.loc[i,'Descripcion Activo'][:7]
    except:
        b=1

for i, n in enumerate(df_cvo['PPU_MMU']):
    try:
        if not (re.match(r'[a-zA-Z0-9]{1,4}-[a-zA-Z0-9]{1,4}$', n)):
            df_cvo.loc[i,'PPU_MMU'] = 'SinNumero'
        else:
            b=2
    except:
        b=1

In [6]:
# Ahora se cargan los archivos auxliares que permiten completar la informacion
df_rs = pd.read_csv('Valores_OT_Asoc_SIC.csv', header =0)
df_bdvehic = pd.read_csv('Clean_Bd_Vehic.csv', encoding='latin-1')
df_cod_org = pd.read_csv('Codigo_Organizacion.csv', sep=';')

df_bdvehic.rename(columns={'MMU/PPU':'PPU_MMU'}, inplace=True)
df_bdvehic['PPU_MMU'] = df_bdvehic['PPU_MMU'].astype(str)
df_bdvehic.drop_duplicates(inplace = True, subset = ['PPU_MMU'])

In [7]:
# Realizo una estandarizacion de los nombres de las columnas

df_rs.columns = df_rs.columns.str.replace(' ', '')
df_cvo.columns = df_cvo.columns.str.replace(' ', '')

df_rs.columns = df_rs.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_cvo.columns = df_cvo.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [8]:
# Se crea el DataFrame que se va a utilizar para completar la informacion, usando los nombres de columnas que se presentan aca
column_names = ['Numero_OT', 'OT_Principal', 'Tipo_OT', 'Estado_OT', 'PPU_MMU', 'Total', 'Desc', 'Marca', 'Modelo', 'Nom', 'Ele_Comp', 'Desc_Falla', 'Cod_Org', 'Nom_Org', 'Cod_Taller', 'UAC', 'UR', 'Tipo_Mant', 'Tipo_Comb', 'Sistema', 'Fec_Creacion', 'Fec_Final', 'Fec_Liberacion', 'Fec_Cierre', 'Year', 'Fabricat_Year']

df = pd.DataFrame(columns = column_names)
df.columns = df.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df.columns = df.columns.str.replace(' ', '')

In [9]:
# Copio la informacion del DataFrame principal a al DataFrame final
df[['Numero_OT', 'OT_Principal', 'Tipo_OT', 'Estado_OT', 'PPU_MMU', 'Ele_Comp', 'Desc_Falla', 'Cod_Org', 'Cod_Taller', 'Tipo_Mant', 'Sistema', 'Fec_Solicitud' ,'Fec_Creacion', 'Fec_Final', 'Fec_Liberacion', 'Fec_Cierre']] = df_cvo[['NumeroOt', 'NumeroOtPrincipal', 'TipoOt', 'Estado', 'PPU_MMU', 'GrupoDeActivos', 'DescripcionOt', 'NombreUR', 'CodigoTaller', 'TipoMantenimiento', 'Sistema', 'Fechasolicitud','FechaCreacion', 'FechaFinalizacion', 'FechaLiberacion', 'FechaCierre']]
df['PPU_MMU'] = df['PPU_MMU'].astype(str)

In [10]:
# Se completa la informacion de la columna 'Tipo OT' para que solo contenga las opciones 'Principal' o 'Secundaria'
df['Tipo_OT'] = np.where(df['OT_Principal'].str.contains('No Tiene'), 'Principal', df['Tipo_OT'])
df['Tipo_OT'] = np.where(df['OT_Principal'].str.contains('OT'), 'Secundaria', df['Tipo_OT'])

mp = {'MPI', 'MPC'}
df.loc[df.Tipo_Mant.isin(mp), 'Tipo_OT'] = 'Principal'

In [11]:
# Esta funcion permite combinar la informacion de los archivos auxiliares, para que se pueda completar la informacion del DataFrame final, la hice como funcion ya que se repite un par de veces
def right_join_test(left_db, right_db, left_col, left_merge_cols, right_col, right_merge_cols):
    left_temp_df = left_db.copy()
    left_temp_df[left_col] = left_temp_df[left_col].astype(str)
    right_temp_df = right_db[right_col].astype(str)
    right_db[right_merge_cols] = pd.merge(left_temp_df, right_temp_df, left_on=left_col, right_on=right_col, how='right', indicator=True)[left_merge_cols]

In [12]:
# Usando la funcion, combino la informacion del archivo auxiliar que contiene la informacion de los vehiculos con la informacion del DataFrame principal
left_cols = ['Descripción', 'Marca', 'Modelo', 'UAC', 'UR', 'Tipo Patente', 'Combustible', 'Función', 'Año Fabricación']
right_cols = ['Desc', 'Marca', 'Modelo', 'UAC', 'UR', 'Nom', 'Tipo_Comb', 'Ele_Comp', 'Fabricat_Year']
right_join_test(df_bdvehic, df, 'PPU_MMU', left_cols, 'PPU_MMU', right_cols)
df['Ele_Comp'].replace(np.nan, 'NO APLICA', inplace=True)

In [13]:
# Ahora se vuelve a usar la funcion, esta vez para valorizar las OTs, con la informacion del reporte de SIC asociadas a OT
left_cols = ['Total']
right_cols = ['Total']
right_join_test(df_rs, df, 'NroOt', left_cols, 'Numero_OT', right_cols)

In [14]:
# Finalmente, usando el codigo de organización, le asigno nombres a las organizaciones, para que no aparezcan como código y sean mas entendibles
left_cols = 'Nombre'
right_cols = 'Nom_Org'
right_join_test(df_cod_org, df, 'Codigo', left_cols, 'Cod_Org', right_cols)

In [15]:
# Como preparación para usar las fechas como formato 'DateTime', hago la siguiente conversion de los meses, ya que estos son los que se reconocen al usar 'esp' como locale
df['Fec_Solicitud'].replace('ENE', 'ene.', regex=True, inplace=True)
df['Fec_Solicitud'].replace('FEB', 'feb.', regex=True, inplace=True)
df['Fec_Solicitud'].replace('MAR', 'mar.', regex=True, inplace=True)
df['Fec_Solicitud'].replace('ABR', 'abr.', regex=True, inplace=True)
df['Fec_Solicitud'].replace('MAY', 'may.', regex=True, inplace=True)
df['Fec_Solicitud'].replace('JUN', 'jun.', regex=True, inplace=True)
df['Fec_Solicitud'].replace('JUL', 'jul.', regex=True, inplace=True)
df['Fec_Solicitud'].replace('AGO', 'ago.', regex=True, inplace=True)
df['Fec_Solicitud'].replace('SEP', 'sep.', regex=True, inplace=True)
df['Fec_Solicitud'].replace('OCT', 'oct.', regex=True, inplace=True)
df['Fec_Solicitud'].replace('NOV', 'nov.', regex=True, inplace=True)
df['Fec_Solicitud'].replace('DIC', 'dic.', regex=True, inplace=True)

In [16]:
# Aca convierto las fechas a formato 'DateTime', las fechas en el ciclo 'for' tienen un formato identico, por lo que es posible convertirlas de una sola vez, la 'Fec_Solicitud' viene con otro formato, por lo que se convierte de manera solitaria
for i in df[['Fec_Creacion', 'Fec_Final', 'Fec_Liberacion', 'Fec_Cierre']]:
    df.loc[:, i] = pd.to_datetime(df.loc[:, i], errors='coerce', format='%Y/%m/%d')
    
df['Fec_Solicitud'] = pd.to_datetime(df['Fec_Solicitud'], errors='coerce', format='%d-%b-%y')

In [17]:
# Como ultimo paso, se selecciona la fecha de la OT, como la minima entre todas las fechas en el DataFrame
df_temp = df[[ 'Fec_Creacion', 'Fec_Final', 'Fec_Liberacion', 'Fec_Cierre']]
df_temp.replace(np.datetime64('NaT'), str(pd.to_datetime('today').normalize().year), inplace=True)
df['Year'] = [np.amin(np.array([row.Fec_Creacion.year,row.Fec_Final.year,row.Fec_Liberacion.year,row.Fec_Cierre.year])) for _, row in df_temp.iterrows()]

C:\Users\Lanzet\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:5233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [18]:
out_file = Path.cwd() / "output.xlsx"
writer = pd.ExcelWriter(out_file, engine = 'xlsxwriter', datetime_format='yyyy-mm-dd', date_format='yyyy-mm-dd')
df.to_excel(writer, index = False)
writer.save()